In [ ]:
import csv
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
from sklearn.decomposition import PCA

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
#embeddings creation
#load GloVE embeddings

def load_glove_embeddings(file_path: str) -> "dict[str, np.ndarray]":
    """
    Loads trained GloVe embeddings downloaded from:
        https://nlp.stanford.edu/projects/glove/
    """
    word_to_embedding = {}
    with open(file_path, "r") as f:
        for line in f:
            word, raw_embeddings = line.split()[0], line.split()[1:]
            embedding = np.array(raw_embeddings, dtype=np.float64)
            word_to_embedding[word] = embedding
    return word_to_embedding

In [ ]:
!unzip '/content/gdrive/MyDrive/COMP 599 Project/glove.6B.zip'

Archive:  /content/gdrive/MyDrive/COMP 599 Project/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
glove_embeddings = load_glove_embeddings("/content/glove.6B.300d.txt") 

In [ ]:
print(len(glove_embeddings))

400000


In [ ]:
from scipy import spatial
def find_closest_embeddings(embedding): 
    return sorted(glove_embeddings.keys(), key=lambda word: spatial.distance.euclidean(glove_embeddings[word], embedding))

In [ ]:
# embedding = glove_embeddings[''] + glove_embeddings['democrat'] - glove_embeddings['republican'] 
# closest = find_closest_embeddings(embedding)
# print(closest[0:20])

In [ ]:
political_attribute_words = [ ["democrat","republican"] , ["democrats","republicans"]  , ["liberal","conservative"] , ["liberals","conservatives"] , 
                             ["obama","mccain"] , ["kerry","bush"] , ["liberalism","conservatism"] , ["communism","capitalism"] , ["refugee","illegal"] ,
                             ["refugees","illegals"] , ["blue","red"] , ["globalist","isolationist"] , ['globalism','isolationism'] , ['obama','trump'] , 
                             ['hillary' , 'trump']]

In [ ]:
def compute_political_subspace(
    word_to_embedding: "dict[str, np.array]",
    political_attribute_words: "list[list[str, str]]",
    n_components: int = 1,
) -> np.array:
    pol_emb = list() 
    for x in political_attribute_words:
        pol_emb.append([word_to_embedding[x[0]],word_to_embedding[x[1]]])
    for l in pol_emb:
        mean = (l[0]+l[1])/2
        l[0] = l[0] - mean
        l[1] = l[1] - mean
    pol_emb = np.array(pol_emb)
    pol_emb = pol_emb.reshape(-1, pol_emb.shape[-1])
    pca = PCA(n_components)
    pca.fit(pol_emb)
    return pca.components_


def project(a: np.array, b: np.array) -> "tuple[float, np.array]":
    numer = np.dot(a,b)
    denom = np.dot(b,b)
    scalar = np.divide(numer,denom)
    vector_projection = np.multiply(scalar,b)
    return scalar, vector_projection

def debias_word_embedding(
    word: str, word_to_embedding: "dict[str, np.array]", political_subspace: np.array
) -> np.array:
    word_emb = word_to_embedding[word]
    for i in range(political_subspace.shape[0]):
            s, projection = project(word_emb,political_subspace[i])

    debiased_embedding = np.subtract(word_emb,projection)

    return debiased_embedding

def hard_debias(
    word_to_embedding: "dict[str, np.array]",
    political_attribute_words: "list[list[str]]",
    n_components: int = 1) -> "dict[str, np.array]":
    gender_subspace = compute_political_subspace(word_to_embedding, political_attribute_words, n_components)
    all_words = list(word_to_embedding.keys())
    debiased_embeddings = list()
    for word in all_words:
        debiased_embeddings.append(debias_word_embedding(word,word_to_embedding,gender_subspace))
   
    dicts = {}
    for i,x in enumerate(all_words):
        dicts[x] = debiased_embeddings[i]
    
    return dicts

In [ ]:
political_subspace = compute_political_subspace(glove_embeddings,political_attribute_words)

In [ ]:
debiased_embeddings = hard_debias(glove_embeddings, political_attribute_words)

In [ ]:
debiased_embeddings_tolist = {k:v.tolist() for k,v in debiased_embeddings.items()}

In [ ]:
import json

with open('/content/gdrive/MyDrive/COMP 599 Project/debiased_embeddings.txt', 'w') as file:
     file.write(json.dumps(debiased_embeddings_tolist)) # use `json.loads` to do the reverse

In [ ]:
print(debiased_embeddings['obama'].tobytes())

b'+\xdd-\xc4\xcab\xbc?V/4_\xeb\xde\xe1?,??S\xd7\x02\xc7\xbf?\x14[\xe4,\xbf\xd4?!*\x7f\xe6\xff\xa3\xe7\xbf\xeaX\xc3\xda)\x03\xd0?F\xc9b\xfc\xd9\x90\xc0?2?\xe7\xcd\x07\x06\xd6\xbf\xebe\x89v\xe2H\xb7?\x90E@\xd0P\x0b\xf1\xbf\x00p}\xa0\xd2\x08>\xbfT5_\xbbI\xaa\xb7?D6i\x0f\xe2\x83\xc9?\xe0\xef\xfb\xf9F,y\xbf\x9cj\xf4I\xbd9\xd6?\xc2\xae\xac5Y~\xe4?\xfb\xaf\xea\xeb\xf0!\xe0\xbf\x90\xe0P\xda\xd7\xb2\xcb\xbf\xec@\xba\xb6e\xa3\xcb?\xf0I7\xbb\xb5c\xcc?f15\x08\xc9\x11\xc7\xbf\xa9\xfb\x1f\xd7\xa2\xa9\xb3\xbf\xa8\xa8\x86\x1c\xde\x9f\xb8?hB#"\xdb\xea\xd7\xbfb\x1a\xc4\xf2\x1a\x7f\xb8\xbf5\x85\xa6J\xa2T\xc0?\xa99\x16\x0eu|\xd3\xbf\xa0\xcd\\\xcb\xa1\xc6\xe0?\x18v\x08\x14\xae\xc3\xa7\xbfuOC\xf6\x1aX\xd5\xbf\x11ao\xaa\xc5d\xe8?\x08\xe2@\x88\xd40\xa7\xbf\xff\xa4~\xe9oR\xc7\xbf\xae\x19!;_\x8e\xc7\xbfss\xf1\xffj\xe7\xf9\xbfC\x1b\xe3l\x86\x8b\x8d?6\xa7i\x82\x9e\x0c\xc8\xbf\x8a\xfb\x1f5\t;\xda\xbff=\x92n\xf0|\xc4\xbf\x04\xde\x8er\x19\xd1\xe1\xbf\x10\x0c\x06\xcc(\x88\xc8\xbf\xa07\x9c\xf9\xa2/\x98?ZN\x94\xadn\x11

In [ ]:
word_to_embedding = {}
with open('/content/gdrive/MyDrive/COMP 599 Project/debiased_embeddings.txt', "w") as f:
  for num in debiased_embeddings.values():
